In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")



dfTest=pd.read_csv("Test_data.csv")
dfTrain=pd.read_csv("Train_data.csv")

In [2]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Codificar variables categóricas
le = LabelEncoder()
dfTrain['protocol_type'] = le.fit_transform(dfTrain['protocol_type'])
dfTrain['service'] = le.fit_transform(dfTrain['service'])
dfTrain['flag'] = le.fit_transform(dfTrain['flag'])

dfTest['protocol_type'] = le.fit_transform(dfTest['protocol_type'])
dfTest['service'] = le.fit_transform(dfTest['service'])
dfTest['flag'] = le.fit_transform(dfTest['flag'])

# Normalizar variables numéricas
scaler = StandardScaler()
X_train = scaler.fit_transform(dfTrain.drop('class', axis=1))
y_train = dfTrain['class']
X_test = scaler.transform(dfTest)

# Modelo sin reducción de dimensionalidad
clf_no_pca = LogisticRegression()
clf_no_pca.fit(X_train, y_train)
y_pred_no_pca = clf_no_pca.predict(X_test)

# Modelo con PCA
pca = PCA(n_components=10)
clf_pca = Pipeline([('pca', pca), ('classifier', LogisticRegression())])
clf_pca.fit(X_train, y_train)
y_pred_pca = clf_pca.predict(X_test)

# Métricas
accuracy_no_pca = accuracy_score(y_train, clf_no_pca.predict(X_train))
precision_no_pca = precision_score(y_train, clf_no_pca.predict(X_train), average='weighted')
recall_no_pca = recall_score(y_train, clf_no_pca.predict(X_train), average='weighted')

accuracy_pca = accuracy_score(y_train, clf_pca.predict(X_train))
precision_pca = precision_score(y_train, clf_pca.predict(X_train), average='weighted')
recall_pca = recall_score(y_train, clf_pca.predict(X_train), average='weighted')

print("Métricas sin PCA:")
print(f"Accuracy: {accuracy_no_pca}, Precision: {precision_no_pca}, Recall: {recall_no_pca}")

print("Métricas con PCA:")
print(f"Accuracy: {accuracy_pca}, Precision: {precision_pca}, Recall: {recall_pca}")


Métricas sin PCA:
Accuracy: 0.9541521117815179, Precision: 0.9542597725502656, Recall: 0.9541521117815179
Métricas con PCA:
Accuracy: 0.9500238170847888, Precision: 0.9501761838124776, Recall: 0.9500238170847888


El modelo sin PCA tiene una precisión ligeramente mayor que el modelo con PCA. La precisión es la proporción de predicciones correctas entre el número total de predicciones realizadas. En este caso, el modelo sin PCA predijo correctamente el 95.41% de los casos, mientras que el modelo con PCA predijo correctamente el 95.00% de los casos.

La precisión también es ligeramente mayor en el modelo sin PCA que en el modelo con PCA. La precisión es una medida de cuán precisas son las predicciones positivas del modelo. En otras palabras, de todos los casos que el modelo predijo como positivos, ¿cuántos realmente lo fueron? En este caso, la precisión ponderada del modelo sin PCA es del 95.42%, mientras que la precisión ponderada del modelo con PCA es del 95.01%.

El recall es igualmente un poco más alto en el modelo sin PCA en comparación con el modelo con PCA. El recall es una medida de la capacidad del modelo para encontrar todos los casos positivos en el conjunto de datos. En otras palabras, de todos los casos positivos en el conjunto de datos, ¿cuántos fueron identificados correctamente por el modelo? En este caso, el recall ponderado del modelo sin PCA es del 95.41%, mientras que el recall ponderado del modelo con PCA es del 95.00%.


In [4]:
from sklearn.model_selection import cross_val_score

# Validación cruzada para el modelo sin PCA
cv_scores_no_pca = cross_val_score(clf_no_pca, X_train, y_train, cv=5, scoring='accuracy')
mean_accuracy_no_pca = cv_scores_no_pca.mean()

# Validación cruzada para el modelo con PCA
cv_scores_pca = cross_val_score(clf_pca, X_train, y_train, cv=5, scoring='accuracy')
mean_accuracy_pca = cv_scores_pca.mean()

print(f"Mean accuracy sin PCA: {mean_accuracy_no_pca}")
print(f"Mean accuracy con PCA: {mean_accuracy_pca}")


Mean accuracy sin PCA: 0.9539535174665005
Mean accuracy con PCA: 0.9362099010016433


La precisión promedio del modelo sin PCA es aproximadamente del 95.40%, mientras que la precisión promedio del modelo con PCA es aproximadamente del 93.62%. La diferencia en la precisión promedio entre los dos modelos es de aproximadamente el 1.78%.

Esto sugiere que el modelo sin PCA tiene un rendimiento ligeramente mejor que el modelo con PCA en términos de precisión promedio. La reducción de dimensionalidad mediante PCA parece haber afectado el rendimiento del modelo en este caso, ya que el modelo con PCA tiene una precisión promedio más baja en comparación con el modelo sin PCA.

Sin embargo, es importante tener en cuenta que la diferencia en la precisión promedio no es muy grande. El modelo con PCA puede seguir siendo útil en situaciones en las que se desee reducir el tiempo de entrenamiento o la complejidad del modelo, especialmente si la disminución en el rendimiento es aceptable para el caso de uso específico.